In [929]:
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
import requests
import json 
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time 
from datetime import datetime,timedelta
from scipy.stats import norm
from tabulate import tabulate

In [930]:
# IF YOU ARE SELECTING NIFTY ,BANKNIFTY,MIDCAP  SELECT SEW[0] FOR INDICES ELSE FOR STOCK SELECT SWG[1] 
symbol="NIFTY"
select_swg=["indices","equities"]
seg=select_swg[0]
EX="29-Feb-2024" #enter the expiry date 

In [931]:

chrome_options = Options()


headers = {
    ":authority":"www.nseindia.com",
    ":method":"GET",
    ":scheme":"https",
    "Accept":"*/*",
    "Accept-Encoding":"gzip, deflate, br" ,
    "Accept-Language":"en-GB,en-US;q=0.9,en;q=0.8,hi;q=0.7",
    "Referer":"https://www.nseindia.com/option-chain",
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
    
}

for key, value in headers.items():
    chrome_options.add_argument(f"--header={key}:{value}")


driver = webdriver.Chrome(options=chrome_options)


driver.get("https://www.nseindia.com/option-chain")

driver.get(f"https://www.nseindia.com/api/option-chain-{seg}?symbol={symbol}")
#driver.get(f"https://www.nseindia.com/api/option-chain-equities?symbol={symbol}")
json_content = driver.page_source
soup = BeautifulSoup(json_content, 'html.parser')

In [932]:
pre_tag = soup.find('pre')
if pre_tag:
    json_content = pre_tag.string.strip()
    print("works")
    driver.close()
    data_dict = json.loads(json_content)
else:
    print("No <pre> tag found in the HTML content.")

works


In [933]:
a=data_dict['records']['data']

In [934]:
ocdata=[]

In [935]:
for i in a:
    for j,k in i.items():
        if j=="CE" or j=="PE":
            info=k
            info["instrumentTyoe"]=j
            ocdata.append(info)


In [936]:
print(len(ocdata))

1268


In [937]:
df=pd.DataFrame(ocdata)
df.drop(columns=['identifier'],inplace=True)

In [938]:
df=df.sort_values(by='expiryDate')

In [939]:
df.reset_index(drop=True,inplace=True)

In [940]:
df['expiryDate']=pd.to_datetime(df['expiryDate'])
df['expiryDate'] = df['expiryDate'].apply(lambda x: pd.to_datetime(x).to_pydatetime())

In [941]:
df

,strikePrice,expiryDate,underlying,openInterest,changeinOpenInterest,pchangeinOpenInterest,totalTradedVolume,impliedVolatility,lastPrice,change,pChange,totalBuyQuantity,totalSellQuantity,bidQty,bidprice,askQty,askPrice,underlyingValue,instrumentTyoe
0,22850,2024-04-04,NIFTY,0.0,0,0.0,0,0.0,0.0,0.0,0.0,100,0,100,80.10,0,0.00,22211.25,CE
1,20450,2024-04-04,NIFTY,0.0,0,0.0,0,0.0,0.0,0.0,0.0,1500,0,750,3.25,0,0.00,22211.25,PE
2,21900,2024-04-04,NIFTY,0.0,0,0.0,0,0.0,0.0,0.0,0.0,1800,2700,900,722.20,900,754.00,22211.25,CE
3,21900,2024-04-04,NIFTY,0.0,0,0.0,0,0.0,0.0,0.0,0.0,250,2300,200,209.65,50,226.55,22211.25,PE
4,20800,2024-04-04,NIFTY,0.0,0,0.0,0,0.0,0.0,0.0,0.0,3600,0,750,11.25,0,0.00,22211.25,PE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1263,19000,2026-12-31,NIFTY,65.0,0,0.0,0,0.0,0.0,0.0,0.0,950,50,600,107.40,50,450.00,22211.25,PE
1264,23000,2026-12-31,NIFTY,124.0,0,0.0,0,0.0,0.0,0.0,0.0,850,100,500,779.45,50,1399.85,22211.25,PE
1265,25000,2026-12-31,NIFTY,0.0,0,0.0,0,0.0,0.0,0.0,0.0,600,0,250,675.05,0,0.00,22211.25,PE
1266,22000,2026-12-31,NIFTY,2.0,0,0.0,0,0.0,0.0,0.0,0.0,850,0,500,519.20,0,0.00,22211.25,PE


In [942]:
df['expiryDate']==EX

0       False
1       False
2       False
3       False
4       False
        ...  
1263    False
1264    False
1265    False
1266    False
1267    False
Name: expiryDate, Length: 1268, dtype: bool

In [943]:
df[df['instrumentTyoe']=='CE']

,strikePrice,expiryDate,underlying,openInterest,changeinOpenInterest,pchangeinOpenInterest,totalTradedVolume,impliedVolatility,lastPrice,change,pChange,totalBuyQuantity,totalSellQuantity,bidQty,bidprice,askQty,askPrice,underlyingValue,instrumentTyoe
0,22850,2024-04-04,NIFTY,0.0,0,0.000000,0,0.00,0.00,0.00,0.000000,100,0,100,80.10,0,0.00,22211.25,CE
2,21900,2024-04-04,NIFTY,0.0,0,0.000000,0,0.00,0.00,0.00,0.000000,1800,2700,900,722.20,900,754.00,22211.25,CE
7,23500,2024-04-04,NIFTY,0.0,0,0.000000,2,13.10,81.45,-22.85,-21.907958,4700,2700,50,58.05,900,75.10,22211.25,CE
8,23100,2024-04-04,NIFTY,0.0,0,0.000000,0,0.00,0.00,0.00,0.000000,900,900,900,128.20,900,154.05,22211.25,CE
9,23950,2024-04-04,NIFTY,0.0,0,0.000000,0,0.00,0.00,0.00,0.000000,7550,1000,1500,1.60,50,65.00,22211.25,CE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,21650,2024-02-29,NIFTY,890.0,-70,-7.291667,222,17.34,615.00,2.30,0.375388,32300,28250,250,615.75,100,619.00,22211.25,CE
1226,21750,2024-02-29,NIFTY,1242.0,69,5.882353,535,14.71,526.45,-0.65,-0.123316,34150,30350,600,524.95,50,527.75,22211.25,CE
1228,21700,2024-02-29,NIFTY,12811.0,-1024,-7.401518,4912,13.32,564.50,-0.70,-0.123850,38000,33850,100,570.30,450,572.45,22211.25,CE
1262,19000,2026-12-31,NIFTY,4.0,0,0.000000,0,0.00,0.00,0.00,0.000000,50,100,50,4532.00,50,7300.00,22211.25,CE


In [944]:
df[df['instrumentTyoe']=='PE']

,strikePrice,expiryDate,underlying,openInterest,changeinOpenInterest,pchangeinOpenInterest,totalTradedVolume,impliedVolatility,lastPrice,change,pChange,totalBuyQuantity,totalSellQuantity,bidQty,bidprice,askQty,askPrice,underlyingValue,instrumentTyoe
1,20450,2024-04-04,NIFTY,0.0,0,0.0,0,0.00,0.0,0.00,0.000000,1500,0,750,3.25,0,0.00,22211.25,PE
3,21900,2024-04-04,NIFTY,0.0,0,0.0,0,0.00,0.0,0.00,0.000000,250,2300,200,209.65,50,226.55,22211.25,PE
4,20800,2024-04-04,NIFTY,0.0,0,0.0,0,0.00,0.0,0.00,0.000000,3600,0,750,11.25,0,0.00,22211.25,PE
5,23700,2024-04-04,NIFTY,0.0,0,0.0,0,0.00,0.0,0.00,0.000000,100,0,100,913.90,0,0.00,22211.25,PE
6,21950,2024-04-04,NIFTY,0.0,0,0.0,11,14.96,226.4,-23.65,-9.458108,1850,2850,50,222.35,100,238.95,22211.25,PE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1261,24000,2026-12-31,NIFTY,0.0,0,0.0,0,0.00,0.0,0.00,0.000000,600,0,250,500.05,0,0.00,22211.25,PE
1263,19000,2026-12-31,NIFTY,65.0,0,0.0,0,0.00,0.0,0.00,0.000000,950,50,600,107.40,50,450.00,22211.25,PE
1264,23000,2026-12-31,NIFTY,124.0,0,0.0,0,0.00,0.0,0.00,0.000000,850,100,500,779.45,50,1399.85,22211.25,PE
1265,25000,2026-12-31,NIFTY,0.0,0,0.0,0,0.00,0.0,0.00,0.000000,600,0,250,675.05,0,0.00,22211.25,PE


In [945]:
ce_data=df[(df['expiryDate']==EX) & (df['instrumentTyoe']=="CE")]

In [946]:
ce_data.reset_index(drop=True,inplace=True)

In [947]:
ce_data.head(5)

,strikePrice,expiryDate,underlying,openInterest,changeinOpenInterest,pchangeinOpenInterest,totalTradedVolume,impliedVolatility,lastPrice,change,pChange,totalBuyQuantity,totalSellQuantity,bidQty,bidprice,askQty,askPrice,underlyingValue,instrumentTyoe
0,23750,2024-02-29,NIFTY,6387.0,5077,387.557252,20982,19.15,0.80,-0.45,-36.000000,1056550,238850,105000,0.75,48400,0.80,22211.25,CE
1,23850,2024-02-29,NIFTY,14032.0,6791,93.785389,27646,20.26,0.75,-0.35,-31.818182,1353750,322250,77300,0.70,90800,0.75,22211.25,CE
2,23450,2024-02-29,NIFTY,8597.0,6182,255.983437,31334,16.87,1.40,-0.05,-3.448276,436750,134500,22450,1.40,25550,1.45,22211.25,CE
3,23700,2024-02-29,NIFTY,16164.0,8260,104.504049,56598,18.72,0.85,-0.35,-29.166667,2228700,444900,219200,0.80,62950,0.85,22211.25,CE
4,23600,2024-02-29,NIFTY,31586.0,26288,496.187240,177961,17.82,0.95,-0.30,-24.000000,3353950,425400,35250,0.95,182150,1.00,22211.25,CE


In [948]:
ce_data.columns

Index(['strikePrice', 'expiryDate', 'underlying', 'openInterest',
       'changeinOpenInterest', 'pchangeinOpenInterest', 'totalTradedVolume',
       'impliedVolatility', 'lastPrice', 'change', 'pChange',
       'totalBuyQuantity', 'totalSellQuantity', 'bidQty', 'bidprice', 'askQty',
       'askPrice', 'underlyingValue', 'instrumentTyoe'],
      dtype='object')

In [949]:
pe_data=df[(df['expiryDate']==EX) & (df['instrumentTyoe']=="PE")]

In [950]:
pe_data.reset_index(drop=True,inplace=True)

In [951]:
pe_data

,strikePrice,expiryDate,underlying,openInterest,changeinOpenInterest,pchangeinOpenInterest,totalTradedVolume,impliedVolatility,lastPrice,change,pChange,totalBuyQuantity,totalSellQuantity,bidQty,bidprice,askQty,askPrice,underlyingValue,instrumentTyoe
0,23450,2024-02-29,NIFTY,0.0,0,0.000000,0,0.00,0.00,0.00,0.000000,13500,13500,1800,1183.30,1800,1230.15,22211.25,PE
1,23850,2024-02-29,NIFTY,74.0,0,0.000000,1,0.00,1517.95,-192.95,-11.277690,13050,13050,600,1565.70,1150,1636.25,22211.25,PE
2,23900,2024-02-29,NIFTY,6.0,0,0.000000,0,0.00,0.00,0.00,0.000000,10500,10500,600,1613.75,600,1685.20,22211.25,PE
3,23750,2024-02-29,NIFTY,60.0,0,0.000000,0,0.00,0.00,0.00,0.000000,11700,10200,1800,1464.10,300,1553.80,22211.25,PE
4,23700,2024-02-29,NIFTY,47.0,0,0.000000,0,0.00,0.00,0.00,0.000000,13500,14700,1800,1439.65,600,1471.85,22211.25,PE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,21900,2024-02-29,NIFTY,82261.0,7716,10.350795,575528,14.87,47.10,-7.35,-13.498623,346150,455150,250,47.10,3750,47.25,22211.25,PE
103,21600,2024-02-29,NIFTY,71437.0,14794,26.117967,252225,16.30,15.70,-3.70,-19.072165,241150,189200,3700,15.75,200,15.80,22211.25,PE
104,21850,2024-02-29,NIFTY,16953.0,6598,63.718011,180243,15.05,39.40,-7.05,-15.177610,135400,157900,750,39.25,700,39.35,22211.25,PE
105,21750,2024-02-29,NIFTY,25130.0,12011,91.554234,202632,15.54,27.15,-6.15,-18.468468,161200,187500,2900,27.15,6350,27.25,22211.25,PE


In [952]:
ce_data

,strikePrice,expiryDate,underlying,openInterest,changeinOpenInterest,pchangeinOpenInterest,totalTradedVolume,impliedVolatility,lastPrice,change,pChange,totalBuyQuantity,totalSellQuantity,bidQty,bidprice,askQty,askPrice,underlyingValue,instrumentTyoe
0,23750,2024-02-29,NIFTY,6387.0,5077,387.557252,20982,19.15,0.80,-0.45,-36.000000,1056550,238850,105000,0.75,48400,0.80,22211.25,CE
1,23850,2024-02-29,NIFTY,14032.0,6791,93.785389,27646,20.26,0.75,-0.35,-31.818182,1353750,322250,77300,0.70,90800,0.75,22211.25,CE
2,23450,2024-02-29,NIFTY,8597.0,6182,255.983437,31334,16.87,1.40,-0.05,-3.448276,436750,134500,22450,1.40,25550,1.45,22211.25,CE
3,23700,2024-02-29,NIFTY,16164.0,8260,104.504049,56598,18.72,0.85,-0.35,-29.166667,2228700,444900,219200,0.80,62950,0.85,22211.25,CE
4,23600,2024-02-29,NIFTY,31586.0,26288,496.187240,177961,17.82,0.95,-0.30,-24.000000,3353950,425400,35250,0.95,182150,1.00,22211.25,CE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,21800,2024-02-29,NIFTY,21806.0,984,4.725771,12308,14.62,481.35,0.00,0.000000,53300,41950,150,481.75,350,483.15,22211.25,CE
103,21850,2024-02-29,NIFTY,2036.0,-13,-0.634456,2137,14.36,428.90,-7.40,-1.696081,128800,35900,100,437.50,100,439.10,22211.25,CE
104,21650,2024-02-29,NIFTY,890.0,-70,-7.291667,222,17.34,615.00,2.30,0.375388,32300,28250,250,615.75,100,619.00,22211.25,CE
105,21750,2024-02-29,NIFTY,1242.0,69,5.882353,535,14.71,526.45,-0.65,-0.123316,34150,30350,600,524.95,50,527.75,22211.25,CE


In [953]:

def tab(t):
    t=dict(t)
    instruments=list(t.keys())
    value=list(t.values())
    head=['INSTRUMENTS','VALUE']
    data={'instruments':instruments,'value':value}
    table = tabulate(data, headers=head, tablefmt="grid")
    return(print(table))

In [954]:
tab(ce_data.iloc[-1])

+-----------------------+----------------------+
| INSTRUMENTS           | VALUE                |
+=======================+======================+
| strikePrice           | 21700                |
+-----------------------+----------------------+
| expiryDate            | 2024-02-29 00:00:00  |
+-----------------------+----------------------+
| underlying            | NIFTY                |
+-----------------------+----------------------+
| openInterest          | 12811.0              |
+-----------------------+----------------------+
| changeinOpenInterest  | -1024                |
+-----------------------+----------------------+
| pchangeinOpenInterest | -7.401517889410914   |
+-----------------------+----------------------+
| totalTradedVolume     | 4912                 |
+-----------------------+----------------------+
| impliedVolatility     | 13.32                |
+-----------------------+----------------------+
| lastPrice             | 564.5                |
+-------------------

In [955]:
ce_data["impliedVolatility"].iloc[-1]

13.32

In [956]:
def black_scholes(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price

def calculate_option_price(underlying_price, strike_price, time_to_expiration, risk_free_rate, implied_volatility):
    
    option_price = black_scholes(underlying_price, strike_price, time_to_expiration, risk_free_rate, implied_volatility)
    return option_price


underlying_price =ce_data['underlyingValue'].iloc[-1]
strike_price = ce_data['strikePrice'].iloc[-1]
today = datetime.now()
expiry_date=ce_data['expiryDate'].iloc[-1]
time_to_expiration = (expiry_date - today)/timedelta(days=1)/365
risk_free_rate = 0.1
implied_volatility =(ce_data['impliedVolatility'].iloc[-1])/100

option_price = calculate_option_price(underlying_price, strike_price, time_to_expiration, risk_free_rate, implied_volatility)


print("Option Price:", option_price)


Option Price: 553.1231642404382


In [957]:
from py_vollib.black_scholes.greeks.analytical import delta,gamma,rho,theta

In [958]:
underlying_price =ce_data['underlyingValue'].iloc[-1]
strike_price = ce_data['strikePrice'].iloc[-1]
today = datetime.now()
expiry_date=ce_data['expiryDate'].iloc[-1]
time_to_expiration = (expiry_date - today)/timedelta(days=1)/365
risk_free_rate = 0.1
implied_volatility =(ce_data['impliedVolatility'].iloc[-1])/100

In [963]:
d=delta('c',underlying_price,strike_price,time_to_expiration,risk_free_rate,implied_volatility)
d

0.937761034774071

In [964]:
g=gamma('c',underlying_price,strike_price,time_to_expiration,risk_free_rate,implied_volatility)
g

0.00034052207518941514

In [965]:
r=rho('c',underlying_price,strike_price,time_to_expiration,risk_free_rate,implied_volatility)
r

3.0018798286438346

In [966]:
t=theta('c',underlying_price,strike_price,time_to_expiration,risk_free_rate,implied_volatility)
t

-9.637968550013069